In [46]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pytesseract
from pytesseract import Output
import os

#INPUT FILE RUN
#Cambiar al archivo que se quiere usar
input_file='Inputdata_TMTA.ipynb'

#Obtiene directorio actual
cwd = os.getcwd()
#Busca directorio arriba
here=os.path.dirname(cwd)
#Arma el directorio donde están los inputs
inputs=os.path.join(here,'INPUTS')
#Arma el camino al archivo
inputs=os.path.join(inputs,input_file)

print('\033[91m' + 'Archivo Input:' + '\033[0m' + inputs)

%run $inputs

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

ntrials=1

for nt in range(ntrials):
    ntrial=nt+1
    
    # Load image, convert to HSV format, define lower/upper ranges, and perform
    # color segmentation to create a binary mask
    #Camino al archivo .png diapositiva
    image = cv2.imread('C:\Ana\Python\Data\Tobii\TMTA\Trial' + str(ntrial) + '.png')

    image1 = cv2.bitwise_not(image)
    gray_image = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    contours, hierarchy = cv2.findContours(threshold_img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    contours_poly = [None] * len(contours)
    # The Bounding Rectangles will be stored here:
    boundRect = []

    # Alright, just look for the outer bounding boxes:
    for i, c in enumerate(contours):

        if hierarchy[0][i][3] == -1:
            contours_poly[i] = cv2.approxPolyDP(c, 3, True)
            boundRect.append(cv2.boundingRect(contours_poly[i]))

    total_boxes=100
    boxes = np.zeros(total_boxes,dtype={"names":('number','x','y','w','h'),'formats':('int','float','float','float','float')})
    # Draw the bounding boxes on the (copied) input image:
    j=0
    for i in range(len(boundRect)):
        color = (0, 255, 0)
        if boundRect[i][3] >40 and boundRect[i][0]!=0:
            cv2.rectangle(image, (int(boundRect[i][0]), int(boundRect[i][1])), \
                  (int(boundRect[i][0] + boundRect[i][2]), int(boundRect[i][1] + boundRect[i][3])), color, 2)
            crop_img = threshold_img[boundRect[i][1]+10:boundRect[i][1]+boundRect[i][3]-10,boundRect[i][0]+10:boundRect[i][0]-10+boundRect[i][2]]
            #cv2.imshow('cropped',crop_img)
            #cv2.waitKey()
            #cv2.destroyAllWindows
            custom_config =  r'--oem 3 --psm 6'

            # now feeding image to tesseract
            details = pytesseract.image_to_string(crop_img, config=custom_config,lang='spa')
            boxes["number"][j]=details
            boxes["x"][j]=boundRect[i][0]
            boxes["y"][j]=boundRect[i][1]
            boxes["w"][j]=boundRect[i][2]
            boxes["h"][j]=boundRect[i][3]
            j=j+1

    ind=np.asarray(np.where(boxes["x"]==0))
    boxes=np.delete(boxes,ind[0],axis=0)
    boxes=np.sort(boxes, order='number')
    #print(boxes)
    #cv2.imshow("Image",image)
    #cv2.waitKey()
    #cv2.destroyAllWindows

    boxes_frame=pd.DataFrame(boxes)
    boxes_frame.to_csv (directory_in + 'TMTA_Boxes_Trial' + str(ntrial) + '.csv',  header=True, index=False, encoding="utf-8-sig")

Archivo Input:C:\Ana\Python\Codigos\INPUTS\Inputdata_TMTA.ipynb
